In [83]:
%matplotlib inline

# Task 1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import  linear_model
import seaborn as sns

import math
np.set_printoptions(precision=3)
plt.rcParams.update({'font.size': 22})
path_X = "../Project2/action.csv"
oX = pd.read_csv(path_X)
path_Y = "../Project2/alert.csv"
oY = pd.read_csv(path_Y)

X = oX.copy()
Y = oY.copy()

In [84]:
attack_names_map = {"Continue":0,"Ping Scan":1,"Install tools":2,"DVWA SQL Injection Exploit":3,"Network service login":4,"Sambacry Explolit":5}
X = X.replace(attack_names_map)

In [85]:
from sklearn.model_selection import train_test_split



X_train, X_test, y_train, y_test = train_test_split(Y, X, test_size=0.30, random_state=0)

In [162]:
traindf = pd.DataFrame(columns=['attack','alert',"timestep"])
count = 0
rows_list = []
for index, row in X_train.iterrows():
    
    for col, value in row.items():
 
        rows_list.append({"alert": value,"attack":y_train.loc[index][col],"timestep":int(col),"index":index,"pred_alert":X_train.loc[index][int(col)-1] if int(col)>0 else 0,"succ_alert": X_train.loc[index][int(col)+1] if int(col)<9 else 0, "pred_alert2":X_train.loc[index][int(col)-2] if int(col)>1 else 0,"succ_alert2":X_train.loc[index][int(col)+2] if int(col)<8 else 0})
        
traindf = pd.DataFrame(rows_list)

testdf = pd.DataFrame(columns=['attack','alert',"timestep"])
count = 0
rows_list = []
for index, row in X_test.iterrows():
    
    for col, value in row.items():
        rows_list.append({"alert": value,"attack":y_test.loc[index][col],"timestep":int(col),"index":index,"pred_alert":X_test.loc[index][int(col)-1] if int(col)>0 else 0,"succ_alert": X_test.loc[index][int(col)+1] if int(col)<9 else 0,"pred_alert2":X_test.loc[index][int(col)-2] if int(col)>1 else 0,"succ_alert2":X_test.loc[index][int(col)+2] if int(col)<8 else 0 })
        
testdf = pd.DataFrame(rows_list)


In [163]:
X_train_norm = traindf[["alert","timestep"]]
Y_train_norm = traindf[["attack"]]

X_test_norm = testdf[["alert","timestep"]]
Y_test_norm = testdf[["attack"]]

X_test_norm

,alert,timestep
0,2237,0
1,1661,1
2,7473,2
3,2815,3
4,2386,4
...,...,...
5995,2176,5
5996,12379,6
5997,1969,7
5998,2125,8


In [71]:
Y_train_norm

,attack
0,0
1,0
2,1
3,5
4,4
...,...
13995,1
13996,3
13997,4
13998,2


In [164]:
from sklearn.ensemble import RandomForestClassifier



clf = RandomForestClassifier(random_state=0)
clf.fit(X_train_norm, Y_train_norm.to_numpy().reshape(14000,))




RandomForestClassifier(random_state=0)

In [166]:
pred = clf.predict(X_test_norm)

pred = pred.reshape(600,10)

In [167]:
Y_test_norm

seq = np.array(Y_test_norm["attack"]).reshape(600,10)




In [168]:
def starting_accuracy(pred,seq):
    right = 0
    total = 0
    for idx,e in enumerate(pred):
        total+=1
        #print(seq[idx],pred[idx])
        for k in range(10):
            if seq[idx].tolist()[k] == 0 and pred[idx].tolist()[k] == 0:
                continue
            else:
                if seq[idx].tolist()[k] != 0 and  pred[idx].tolist()[k] != 0:
                    right+=1
                    break
                else:
                    break
    return right/total

acc_start = starting_accuracy(pred,seq)
acc_start

0.675

In [169]:
def overall_accuracy(pred,seq):
    right = 0
    total = 0
    for idx,e in enumerate(pred):
        
        for k in range(10):
            total+=1
            if pred[idx].tolist()[k] == seq[idx].tolist()[k]:
                right+=1
    return right/total

acc_ov = overall_accuracy(pred,seq)
acc_ov

0.6305

In [171]:
# CHECK FOR OVERFITTING
overfittingX = pd.read_csv("../data2-Testing/action.csv")
attack_names_map = {"Continue":0,"Ping Scan":1,"Install tools":2,"DVWA SQL Injection Exploit":3,"Network service login":4,"Sambacry Explolit":5}
ovX = overfittingX.replace(attack_names_map)
overfittingY = pd.read_csv("../data2-Testing/alert.csv")

testdf = pd.DataFrame(columns=['attack','alert',"timestep"])
count = 0
rows_list = []
for index, row in ovX.iterrows():
    
    for col, value in row.items():
 
        rows_list.append({"alert": value,"attack": ovX.loc[index][col],"timestep":int(col),"index":index,"pred_alert":overfittingY.loc[index][int(col)-1] if int(col)>0 else 0,"succ_alert": overfittingY.loc[index][int(col)+1] if int(col)<9 else 0, "pred_alert2":overfittingY.loc[index][int(col)-2] if int(col)>1 else 0,"succ_alert2":overfittingY.loc[index][int(col)+2] if int(col)<8 else 0})
        
testdf = pd.DataFrame(rows_list)

X_test_norm = testdf[["alert","timestep"]]
Y_test_norm = testdf[["attack"]]

pred = clf.predict(X_test_norm)

pred = pred.reshape(2000,10)

seq = np.array(Y_test_norm["attack"]).reshape(2000,10)

acc_start = starting_accuracy(pred,seq)
acc_ov = overall_accuracy(pred,seq)

print(acc_start,acc_ov)

0.1315 0.38875
